In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from torchinfo import summary

torch.manual_seed(123)
np.random.seed(123)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.set_default_device(device)
print(f"Using {device}")

Using cpu


In [2]:
from bokeh.io import show, output_notebook
from bokeh.models  import TabPanel, Tabs, LegendItem, Legend
from bokeh.plotting import figure, Row
from bokeh.layouts import column

output_notebook()

Loading BokehJS ...

In [3]:
# General Parameters

EPOCHS = 30
BATCH = 128
TEST_SPLIT = 0.2

OVERSAMPLING = 0.4         # Final fraction of positives in the training set

INPUT_SIZE = 30            # Time, Amount and features V1 thought V28
MID_SIZE = 30*30           # single layer common to all models
TARGET_SIZE = 2            # is fraud? 0/1

DATAPATH = "creditcard.csv"

# Data Exploration and Preparation

The data utilized for this project is available [here](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

In [4]:
df = pd.read_csv(DATAPATH)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
t = len(df)
f = df["Class"].sum()

print(f"Total Transactions:\t{ t }")
print(f"Total Frauds:\t\t{ f }")
print(f"Total Not Frauds:\t{ t - f }")
print(f"Fraud Fraction:\t\t{100 * f / t :00000.000}%")

Total Transactions:	284807
Total Frauds:		492
Total Not Frauds:	284315
Fraud Fraction:		0.2%


In [6]:
df=(df-df.min())/(df.max()-df.min())

In [7]:
class PandasDataset(TensorDataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.targets = torch.tensor(targets.values, dtype=torch.long)  # Adjust dtype for regression

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [8]:
X = df.drop(["Class"], axis=1)
y = df["Class"]

# Separate training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, stratify=y, random_state=42)

# Oversampling training set
rebalance = X_train
rebalance["Class"] = y_train
positive = rebalance[rebalance["Class"] == 1]
while (rebalance["Class"] == 1).sum() / len(rebalance) < OVERSAMPLING:
    rebalance= pd.concat([rebalance, positive], ignore_index=True)

X_train = rebalance.drop(["Class"], axis=1)
y_train = rebalance["Class"]

# Convert to torch tensors
training_loader = DataLoader(PandasDataset(X_train, y_train), batch_size=BATCH, shuffle=True)
validation_loader = DataLoader(PandasDataset(X_test, y_test), batch_size=BATCH, shuffle=False)

# Models Implementation

### Multiple Layer Perceptron

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 512)
        self.fc2 = nn.Linear(512, TARGET_SIZE)

    def forward(self, x):
        x = F.relu(self.fc1(x))  
        return self.fc2(x)

In [10]:
summary(MLP())

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Linear: 1-1                            15,872
├─Linear: 1-2                            1,026
Total params: 16,898
Trainable params: 16,898
Non-trainable params: 0

### Convolutional Neural Network

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 900)
        self.conv = nn.Conv2d(
            in_channels=4, 
            out_channels=8, 
            kernel_size=(10, 10)
        )
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=1)
        self.flatten = nn.Flatten()
        self.fc2 = nn.Linear(128, TARGET_SIZE)

    def forward(self, x):
        x = F.relu(self.fc1(x))  
        #print(x.shape)
        x = x.view(-1, 4, 15, 15)
        #print(x.shape)
        x = F.relu(self.conv(x))
        #print(x.shape)
        x = self.pool(x)
        #print(x.shape)
        x = self.flatten(x)
        #print(x.shape)
        return self.fc2(x)

In [12]:
summary(CNN())

Layer (type:depth-idx)                   Param #
CNN                                      --
├─Linear: 1-1                            27,900
├─Conv2d: 1-2                            3,208
├─MaxPool2d: 1-3                         --
├─Flatten: 1-4                           --
├─Linear: 1-5                            258
Total params: 31,366
Trainable params: 31,366
Non-trainable params: 0

### Long-Short Term Memory

In [13]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 900)
        
        # Bidirectional LSTM layers
        self.lstm = nn.LSTM(input_size=30, 
                            hidden_size=30,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        
        self.fc2 = nn.Linear(60, TARGET_SIZE)

    def forward(self, x):
        x = F.relu(self.fc1(x))  
        x = x.view(-1, 30, 30)
        x, (h_n, c_n) = self.lstm(x) 
        return self.fc2(x[:,-1,:])

In [14]:
summary(LSTM())

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─Linear: 1-1                            27,900
├─LSTM: 1-2                              36,960
├─Linear: 1-3                            122
Total params: 64,982
Trainable params: 64,982
Non-trainable params: 0

### Swarm Characteristic Neural Network

In [15]:
class SwarmFilter(nn.Module):
    def __init__(self, units=32):
        super(SwarmFilter, self).__init__()
        self.units = units
        self.filter = nn.Parameter(torch.randn(units))  # Trainable parameter with random initialization

    def forward(self, x):
        # Compute the mean along the last axis, keeping the dimensions
        mean_values = torch.mean(x, dim=-1, keepdim=True)
        return mean_values * self.filter

In [16]:
class SCNN(nn.Module):
    def __init__(self):
        super(SCNN, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 300)
        self.swarm1 = SwarmFilter(units=300)
        self.swarm2 = SwarmFilter(units=100)
        self.fc2 = nn.Linear(100, TARGET_SIZE) 

    def forward(self, x):
        x = F.relu(self.fc1(x))  
        x = self.swarm1(x)     
        x = self.swarm2(x)     
        return self.fc2(x)

In [17]:
summary(SCNN())

Layer (type:depth-idx)                   Param #
SCNN                                     --
├─Linear: 1-1                            9,300
├─SwarmFilter: 1-2                       300
├─SwarmFilter: 1-3                       100
├─Linear: 1-4                            202
Total params: 9,902
Trainable params: 9,902
Non-trainable params: 0

# Running the models

### Implementation

##### Visuals

In [18]:
def progressbar(i, total, start):
    x = int(100*i/total)
    print(f"\t[{ u'█' * x }{ '.' * (100-x) }] { i : 6.0f}/{ total } Waiting for {time.time()-start:5.3f} seconds", end='\r')
    if i == total:
        print()

In [19]:
def display_stats(loss, cm):
    
    # Extract confusion matrix values
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]
    
    # Calculate metrics
    accuracy = (TP + TN) / cm.sum()
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    
    # Display metrics
    print(f'\t\tLoss: {loss}')
    
    print(f'\t\tConfusion Matrix')
    print(f'\t\t   True  Negative:  {int(TN)}')
    print(f'\t\t   False Negative:  {int(FN)}')
    print(f'\t\t   False Positive:  {int(FP)}')
    print(f'\t\t   True  Positive:  {int(TP)}')
    
    print(f'\t\tAccuracy:           {accuracy:0.3f}')
    print(f'\t\tPrecision:          {precision:0.3f}')
    print(f'\t\tSensitivity:        {sensitivity:0.3f}')
    print(f'\t\tSpecificity:        {specificity:0.3f}')

    return accuracy, precision, sensitivity, specificity

##### Training and Validation

In [20]:
def train_epoch(model, optimizer, criterion):
    total_loss = 0.
    confusion = np.zeros((2,2))
    size = len(training_loader)
    start = time.time()
    for i, data in enumerate(training_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Adjust learning weights
        optimizer.step()
        
        # Gather data and report
        preds = outputs.data.max(1)[1]
        total_loss += loss.item()
        confusion += confusion_matrix(labels.detach().numpy(), preds.detach().numpy(), labels=[0,1])
        
    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, confusion

In [21]:
def validate_epoch(model, criterion):
    total_loss = 0.
    confusion = np.zeros((2,2))
    size = len(validation_loader)
    start = time.time()
    for i, data in enumerate(validation_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Make predictions for this batch
        outputs = model(inputs)
        
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
                
        # Gather data and report
        preds = outputs.data.max(1)[1]
        total_loss += loss.item()
        confusion += confusion_matrix(labels.detach().numpy(), preds.detach().numpy(), labels=[0,1])

    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, confusion

##### Data Collection

In [22]:
def run_model(model, label="", color="navy", epochs=EPOCHS):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    stats = {
        "Label": label,
        "Color": color,
        "Training": {
            "Loss": [],
            "Accuracy": [],
            "Precision": [],
            "Sensitivity": [],
            "Specificity": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Validation": {
            "Loss": [],
            "Accuracy": [],
            "Precision": [],
            "Sensitivity": [],
            "Specificity": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Parameters": sum(p.numel() for p in model.parameters()),
    }
    
    train_time = 0
    val_time = 0
    
    for epoch in range(epochs):
        
        # Set the model to training mode
        model.train()  
        
        # Train epoch
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"\tTraining")
        start = time.time()
        train_loss, train_cm = train_epoch(model, optimizer, criterion)
        train_time += time.time() - start
        # Print training stats for the epoch
        train_acc, train_pre, train_sen, train_spe = display_stats(train_loss, train_cm)
        
        # Run validation
        model.eval()
        val_loss = 0.0
        val_cm = np.zeros((2,2))
    
        print(f"\tValidation")
        start = time.time()
        with torch.no_grad():  # Disable gradient calculation during validation
            # Run validation
            val_loss, val_cm = validate_epoch(model, criterion)
        val_time += time.time() - start
        # Print validation stats
        val_acc, val_pre, val_sen, val_spe = display_stats(val_loss, val_cm)

        # save stats
        stats["Training"]["Loss"].append(train_loss)
        stats["Training"]["Accuracy"].append(train_acc)
        stats["Training"]["Precision"].append(train_pre)
        stats["Training"]["Sensitivity"].append(train_sen)
        stats["Training"]["Specificity"].append(train_spe)

        stats["Validation"]["Loss"].append(val_loss)
        stats["Validation"]["Accuracy"].append(val_acc)
        stats["Validation"]["Precision"].append(val_pre)
        stats["Validation"]["Sensitivity"].append(val_sen)
        stats["Validation"]["Specificity"].append(val_spe)

    stats["Training"]["Loss"] = np.array(stats["Training"]["Loss"])
    stats["Validation"]["Loss"] = np.array(stats["Validation"]["Loss"])
    
    stats["Training"]["Time"] = train_time
    stats["Training"]["Time/Epoch"] = train_time / epochs

    stats["Validation"]["Time"] = val_time
    stats["Validation"]["Time/Epoch"] = val_time / epochs
    
    return stats

### Running

In [23]:
mlp_stats = run_model(MLP(), label="MLP", color="green")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for 8.712 seconds
		Loss: 0.16051812238118715
		Confusion Matrix
		   True  Negative:  223999
		   False Negative:  19008
		   False Positive:  3452
		   True  Positive:  132682
		Accuracy:           0.941
		Precision:          0.975
		Sensitivity:        0.875
		Specificity:        0.985
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    446/446 Waiting for 0.887 seconds
		Loss: 0.0953514797554556
		Confusion Matrix
		   True  Negative:  55804
		   False Negative:  8
		   False Positive:  1060
		   True  Positive:  90
		Accuracy:           0.981
		Precision:          0.078
		Sensitivity:        0.918
		Specificity:        0.981
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for 8

In [24]:
cnn_stats = run_model(CNN(), label="CNN", color="navy")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for 12.868 seconds
		Loss: 0.14967255821747633
		Confusion Matrix
		   True  Negative:  223148
		   False Negative:  16968
		   False Positive:  4303
		   True  Positive:  134722
		Accuracy:           0.944
		Precision:          0.969
		Sensitivity:        0.888
		Specificity:        0.981
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    446/446 Waiting for 1.078 seconds
		Loss: 0.030799211123999994
		Confusion Matrix
		   True  Negative:  56481
		   False Negative:  11
		   False Positive:  383
		   True  Positive:  87
		Accuracy:           0.993
		Precision:          0.185
		Sensitivity:        0.888
		Specificity:        0.993
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting fo

In [25]:
lstm_stats = run_model(LSTM(), label="LSTM", color="orange")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for 165.162 seconds
		Loss: 0.1649920960012578
		Confusion Matrix
		   True  Negative:  223320
		   False Negative:  19510
		   False Positive:  4131
		   True  Positive:  132180
		Accuracy:           0.938
		Precision:          0.970
		Sensitivity:        0.871
		Specificity:        0.982
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    446/446 Waiting for 2.948 seconds
		Loss: 0.05763439415231548
		Confusion Matrix
		   True  Negative:  56227
		   False Negative:  10
		   False Positive:  637
		   True  Positive:  88
		Accuracy:           0.989
		Precision:          0.121
		Sensitivity:        0.898
		Specificity:        0.989
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for

In [26]:
scnn_stats = run_model(SCNN(), label="SCNN", color="red")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for 13.306 seconds
		Loss: 0.23460934541986234
		Confusion Matrix
		   True  Negative:  223232
		   False Negative:  27055
		   False Positive:  4219
		   True  Positive:  124635
		Accuracy:           0.918
		Precision:          0.967
		Sensitivity:        0.822
		Specificity:        0.981
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    446/446 Waiting for 1.053 seconds
		Loss: 0.11680390644033393
		Confusion Matrix
		   True  Negative:  55140
		   False Negative:  8
		   False Positive:  1724
		   True  Positive:  90
		Accuracy:           0.970
		Precision:          0.050
		Sensitivity:        0.918
		Specificity:        0.970
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]   2963/2963 Waiting for

# Comparative Analyse

In [27]:
models = [mlp_stats, cnn_stats, lstm_stats, scnn_stats]

### Extracting Visualization

In [28]:
# Define line plots
# Loss
loss = figure(width=800, height=400, title="Loss")
delta_loss = figure(width=800, height=400, title="Overfitting")

delta_loss.yaxis.axis_label = "Val - Train Loss"

# Metrics
accuracy = figure(width=800, height=400, title="Accuracy")
precision = figure(width=800, height=400, title="Precision")
sensitivity = figure(width=800, height=400, title="Sensitivity")
specificity = figure(width=800, height=400, title="Specificity")

guide = [[0], [0], [1], [1], [1], [1]]
for i, fig in enumerate([loss, delta_loss, accuracy, precision, sensitivity, specificity]):
    fig.add_layout(Legend(), 'right')
    fig.line([0], guide[i], line_width=3, legend_label="Training", line_dash="dashed",line_color="black")
    fig.xaxis.axis_label = 'epoch'

# Create auxiliar arrays
x_epoches = np.arange(EPOCHS)

# Plot lines
for stats in models:
    label = stats["Label"]
    color = stats["Color"]
    line_w = 3

    # Plot loss data
    loss.line(x_epoches, stats["Validation"]["Loss"], line_width=line_w, legend_label=label,line_color=color)
    loss.line(x_epoches, stats["Training"]["Loss"], line_width=line_w, line_dash="dashed", line_color=color)

    delta = stats["Validation"]["Loss"] - stats["Training"]["Loss"]
    delta_loss.line(x_epoches, delta, line_width=line_w, legend_label=label, line_color=color)

    # Plot Validation Metrics
    accuracy.line(x_epoches, stats["Validation"]["Accuracy"], line_width=line_w, legend_label=label, line_color=color)
    precision.line(x_epoches, stats["Validation"]["Precision"], line_width=line_w, legend_label=label, line_color=color)
    sensitivity.line(x_epoches, stats["Validation"]["Sensitivity"], line_width=line_w, legend_label=label, line_color=color)
    specificity.line(x_epoches, stats["Validation"]["Specificity"], line_width=line_w, legend_label=label, line_color=color)

    # Plot Training Metrics
    accuracy.line(x_epoches, stats["Training"]["Accuracy"], line_width=line_w, line_color=color, line_dash="dashed")
    precision.line(x_epoches, stats["Training"]["Precision"], line_width=line_w, line_color=color, line_dash="dashed")
    sensitivity.line(x_epoches, stats["Training"]["Sensitivity"], line_width=line_w, line_color=color, line_dash="dashed")
    specificity.line(x_epoches, stats["Training"]["Specificity"], line_width=line_w, line_color=color, line_dash="dashed")

In [29]:
# Prepare histograms
model_labels = []

total_time_values = [[],[]]
epoch_time_values = [[],[]]

parameters = []

best_acc  = [[],[]]
best_pre  = [[],[]]
best_sen  = [[],[]]
best_spe  = [[],[]]

for stats in models:
    label = stats["Label"]
    model_labels.append(label)

    total_time_values[0].append(stats["Training"]["Time"])
    epoch_time_values[0].append(stats["Training"]["Time/Epoch"])
    total_time_values[1].append(stats["Validation"]["Time"])
    epoch_time_values[1].append(stats["Validation"]["Time/Epoch"])


    parameters.append(stats["Parameters"])
    
    best_acc[0].append(max(stats["Training"]["Accuracy"]))
    best_acc[1].append(max(stats["Validation"]["Accuracy"]))
    
    best_pre[0].append(max(stats["Training"]["Precision"]))
    best_pre[1].append(max(stats["Validation"]["Precision"]))
    
    best_sen[0].append(max(stats["Training"]["Sensitivity"]))
    best_sen[1].append(max(stats["Validation"]["Sensitivity"]))
    
    best_spe[0].append(max(stats["Training"]["Specificity"]))
    best_spe[1].append(max(stats["Validation"]["Specificity"]))

In [30]:
# Time related
total_time = figure(x_range=model_labels, width=800, height=400, title="Total Running Time")
epoch_time = figure(x_range=model_labels, width=800, height=400, title="Time per Epoch")

total_time.vbar(x=model_labels, top=total_time_values[0], color="navy", width=0.75,legend_label="Training")
total_time.vbar(x=model_labels, top=total_time_values[1], color="orange", width=0.5,legend_label="Validation")

epoch_time.vbar(x=model_labels, top=epoch_time_values[0], color="navy", width=0.75,legend_label="Training")
epoch_time.vbar(x=model_labels, top=epoch_time_values[1], color="orange", width=0.5,legend_label="Validation")

for fig in [total_time, epoch_time]:
    fig.legend.location = 'top_left'
    fig.yaxis.axis_label = 'seconds'
    fig.y_range.start = 0

In [31]:
# Parameter
param = figure(x_range=model_labels, width=800, height=400, title="Parameters")
param_exp = figure(x_range=model_labels, width=800, height=400, title="Parameters", y_axis_type="log",  y_range = [10 ** 3, 10 ** 5])

param.vbar(x=model_labels, top=parameters, color="navy", width=0.75)
param_exp.vbar(x=model_labels, bottom=[10 ** 0]*len(model_labels), top=parameters, color="navy", width=0.75)

param.y_range.start = 0

In [32]:
# Best performance plots
b_accuracy    = figure(x_range=model_labels, width=800, height=400, title="Best Accuracy")
b_precision   = figure(x_range=model_labels, width=800, height=400, title="Best Precision")
b_sensitivity = figure(x_range=model_labels, width=800, height=400, title="Best Sensitivity")
b_specificity = figure(x_range=model_labels, width=800, height=400, title="Best Specificity")

# Add unified evolution legends
for fig in [b_accuracy, b_precision, b_sensitivity, b_specificity]:
    fig.add_layout(Legend(), 'right')

b_accuracy.vbar(x=model_labels, top=best_acc[0], color="navy", width=0.75,legend_label="Training")
b_accuracy.vbar(x=model_labels, top=best_acc[1], color="orange", width=0.5,legend_label="Validation")
b_accuracy.y_range.start = 0.95

b_precision.vbar(x=model_labels, top=best_pre[0], color="navy", width=0.75,legend_label="Training")
b_precision.vbar(x=model_labels, top=best_pre[1], color="orange", width=0.5,legend_label="Validation")
b_precision.y_range.start = 0

b_sensitivity.vbar(x=model_labels, top=best_sen[0], color="navy", width=0.75,legend_label="Training")
b_sensitivity.vbar(x=model_labels, top=best_sen[1], color="orange", width=0.5,legend_label="Validation")
b_sensitivity.y_range.start = 0.9

b_specificity.vbar(x=model_labels, top=best_spe[0], color="navy", width=0.75,legend_label="Training")
b_specificity.vbar(x=model_labels, top=best_spe[1], color="orange", width=0.5,legend_label="Validation")
b_specificity.y_range.start = 0.985

### Visualizing

In [33]:
# Increase font size and make them all bold
for fig in [
    param, param_exp,
    loss, delta_loss,
    total_time, epoch_time,
    accuracy, b_accuracy,
    precision, b_precision,
    sensitivity, b_sensitivity,
    specificity, b_specificity,
]:
    fig.title.text_font_size = "15pt"
    fig.title.padding = 5
    
    fig.yaxis.major_label_text_font_size = "12pt"
    fig.yaxis.major_label_text_font_style = "bold"
    fig.yaxis.axis_label_text_font_size = "12pt"
    fig.yaxis.axis_label_text_font_style = "bold"
    
    fig.xaxis.major_label_text_font_size = "12pt"
    fig.xaxis.major_label_text_font_style = "bold"
    fig.xaxis.axis_label_text_font_size = "12pt"
    fig.xaxis.axis_label_text_font_style = "bold"

In [34]:
# Create tabs using TabPanel
tabs = Tabs(tabs=[
    TabPanel(child=column(param, param_exp), title="Size"),
    TabPanel(child=column(loss, delta_loss), title="Loss"),
    TabPanel(child=column(total_time, epoch_time), title="Time"),
    TabPanel(child=column(accuracy, b_accuracy), title="Accuracy"),
    TabPanel(child=column(precision, b_precision), title="Precision"),
    TabPanel(child=column(sensitivity, b_sensitivity), title="Sensitivity"),
    TabPanel(child=column(specificity, b_specificity), title="Specificity"),
])

# Show the tabs
show(tabs)